In [31]:
import torch
import torchvision
import numpy as np
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
import sys
from tqdm import tqdm

sys.path.append("/workspaces/ood/")

%load_ext autoreload
%autoreload 2
device = torch.device("cuda")
torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [22]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

In [23]:
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)

In [8]:
wide_resnet50 = torchvision.models.wide_resnet50_2(pretrained=False).to(device)
wide_resnet50.load_state_dict(torch.load('/workspaces/ood/data/models/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth'))

<All keys matched successfully>

In [27]:
from fastflow.extract_features import create_feature_dataset

model_name = 'wide_resnet50'
dataset_name = 'cifar10train'
layers = ['layer2', 'layer3', 'layer4']
outname = '../data/feature_maps/' + '_'.join([dataset_name] + [model_name] + layers) + '.npy'

create_feature_dataset(model=wide_resnet50, layers=layers, dataset=cifar_data_train, out_name=outname)

In [30]:
from feature_extractor import FeatureExtractor
encoder = FeatureExtractor(wide_resnet50, layers)
encoder.eval();

In [ ]:
for image, image_cls in tqdm(cifar_data_train):
    features = encoder(torch.unsqueeze(image.to(device), dim=0))
    preds.append(pred[:, :, 0, 0])
    image_clses.append(image_cls)